### 图像特征提取
- 应用：图像拼接、图像匹配
- 特征检测和提取算法：Harris（检测角点）SIFT（检测斑点blob）SURF（检测斑点）FAST（检测角点）BRIEF（检测斑点）ORB（带方向的FAST算法与具有旋转不变性的BRIEF算法）
- 特征匹配算法：暴力匹配（Brute-Force）基于FLANN匹配。
- 特征：特殊的图形区域、独特性和易于识别性--角点和高密度区域。大量重复区域和低密度区域不适合作为特征，边缘时很好的特征，斑点（与周围有很大差别的区域）

### 各类算法对比
#### FAST

    原理简单
    执行速度相当快
    主要用于侦测corners，亦可侦测blob。
    适用于要求速度的real-time analysis
    适用于速度慢或较低阶的执行环境。
    使用度极高，尤其在需要即时的realtime环境。

    fast = cv2.FastFeatureDetector()

    
#### Harris

    速度相当快，但仍较FAST慢，不过侦测corner比起FAST准确一些。
    广泛应用于侦测edges及corners

    dst = cv2.cornerHarris(gray,2,3,0.04)
    
##### cv2.cornerHarris() 
    - img： 数据类型为 ﬂoat32 的入图像
    - blockSize： 角点检测中指定区域的大小
    - ksize： Sobel求导中使用的窗口大小 
    - k： 取值参数为 [0,04,0.06]
    
#### GFTT

    GFTT为Harris keypoint detector的再修正。
    针对edges/corner detect的准确率与速度有所提升，但实际上使用感觉差异不大。

    detector = cv2.FeatureDetector_create(“GFTT")
    kps = detector.detect(gray)

#### DoG (SIFT)

    原理较为复杂。
    主要针对blob 侦测，不过亦可侦测corners。
    可适应物件的scale(大小变化)及angle（旋转角度）等情况。
    在DoG模型中，使用了不同尺寸影像并套用高斯模糊，比较不同模糊比例之间的变化，来决定是否为keypoint。
    由于计算量大，DoG的执行速度较慢，不适用于realtime的环境。
    SIFT已广泛的应用于电脑视觉领域，并成为评定新keypoint detecter的效率指标。

    sift = cv2.SIFT()

#### Fast Hessian (SURF)

    基于DoG（SIFT）模型而发展，改善其速度慢的缺点。
    可使用于realtime的环境，但速度仍比不上FAST等keypoint detector。
    与DoG相同，用于侦测corners以及明显材质纹路的textures。
    
    
#### STAR

    使用于侦测blog area。
    与DoG相同，亦使用高斯模糊计算，但由于其演算方式相当复杂，目前并不常被使用。
    侦测速度相当快，可应用于realtime分析，但效果并不如FAST。
    
#### MSER

    1. 使用于侦测blog area

    2. 判断的依据（需满足此三点），则认定为blog area：

        a. 相连的像素

        b. 相似的像素强度

        c. 与背景成对比

    3. 对于小区域且对比明显的area侦测效果良好。

    4. 不适用分析模糊的影像。

    5. 不适合使用于要求速度的realtime环境。

    6. 输入的影像符合其要求的三种条件，则侦测速度可加快。

#### Dense

    Dense是最简单的keypoint detector。
    它的原理是先设定一个K值，然后均匀的在整张图面上以每隔K pixels的距离布置一个keypoint，因此严格来说，Dense并没有detect的动作。
    在某些情况下（如风景类图片），Dense的效果其实不会比FAST 、Harris 、 DoG等其它detector差。
    Dense算法虽然简单，但也是其缺点，因为过于简单而产生过多的keypoints，甚至于超过图片中实际的数量，过多的keypoints需要更多的记忆体、储存及运算量。
    单纯的Dense无法适用于不同尺寸的图像，因此需在Keypoints上加入不同半径大小的侦测点(Multiple radii)才能应用不同的尺寸大小(scale invariant)。
    资源耗用大，因此Dense不适用于realtime及运行于小型装置。
    适合用于机器学习的影像分类及基于内容的影像搜索。
    Dense分为单一radii及多重radii两种方法，其中后者可适应尺寸变化的情况。


#### BRISK

    FAST keypoint detectorh的强化，主要是针对尺寸缩放的部份（scale space invariance）。
    会先以pyramid方式产生尺寸大小不同的影像，再针对每一层进行FAST运算及取值。
    速度快，适用于realtime的环境。
    适用于侦测Corners。


#### ORB

    与BRISK相同，ORB亦是针对FAST的强化，但除了scale space invariance，亦加入了旋转不变性(rotation invariance).。
    ORB原理有三大步骤：
    Pyramid图像尺寸并进行各尺寸的FAST计算。
    使用Harris keypoint detector的方法计算每个keypoint分数），并进行排序，最多仅取500个keypoints，其余则丢弃。
    于此第三步中加入旋转不变性，使用「intensity centroid」计算每个keypoint的rotation。
    ORB与BRISK相同，继承了FAST运算快速的特性，可适用于realtime分析。
    
    orb = cv2.ORB()

#### opencv 3.3之后不再免费开发SIFT/SURF

### 特征匹配
在影像中选取重要的特征点，接着以其为base取得周围的特征（即local features），这些来自不同相片的local features 会通过Feature matching功能来比对是否有相同的物件。
使用 OpenCV 中的蛮力（Brute-Force）匹配和 FLANN 匹配

蛮力匹配器是很简单的。首先在第一幅图像中选取一个关键点然后依次与第二幅图像的每个关键点进行（描述符）距离测试，最后返回距离最近的关键点。

FLANN 是快速最近邻搜索包（Fast_Library_for_Approximate_Nearest_Neighbors）的简称。它是一个对大数据集和高维特征进行最近邻搜索的算法的集合，而且这些算法都已经被优化过了。在面对大数据集时它的效果要好于 BFMatcher。


In [3]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def cv_show(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
img1 = cv2.imread('img/box.png', 0)
img2 = cv2.imread('img/box_in_scene.png', 0)

In [6]:
cv_show('img1',img1)

In [7]:
cv_show('img2',img2)

In [8]:
#opencv 3.3之后不再免费开发SIFT
orb = cv2.FastFeatureDetector()

In [ ]:
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

In [ ]:
# crossCheck表示两个特征点要互相匹，例如A中的第i个特征点与B中的第j个特征点最近的，并且B中的第j个特征点到A中的第i个特征点也是 
#NORM_L2: 归一化数组的(欧几里德距离)，如果其他特征计算方法需要考虑不同的匹配计算方式
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

1对1的匹配

In [ ]:
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

In [ ]:
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None,flags=2)

In [ ]:
cv_show('img3',img3)

k对最佳匹配

In [ ]:
bf = cv2.BFMatcher()

matches = bf.knnMatch(des1, des2, k=2)

In [ ]:
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

In [ ]:
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=2)

In [ ]:
cv_show('img3',img3)